# Task
We want to achieve the following workflow:

1. do addition until reaching a certain value

**Note:**

**This might seem simple for a python program. But you can imagine, we can swap 'do addition' to something else, like machine learning training.**

**Run through this python notebook and think about the differences between the while loop and dflow recurse steps.**

In [2]:
from dflow import (
    InputParameter,
    Inputs,
    OutputParameter,
    ShellOPTemplate,
    Step,
    Steps,
    Workflow,
)

In [3]:
plus1 = ShellOPTemplate(
    name="plus1",
    image="alpine:3.15",
    script="echo 'This is iter {{inputs.parameters.iter}}' && "
    "echo $(({{inputs.parameters.iter}}+1)) > /tmp/result.txt",
)
plus1.inputs.parameters = {"iter": InputParameter()}
plus1.outputs.parameters = {"iter": OutputParameter(value_from_path="/tmp/result.txt")}

In [4]:
steps = Steps(
    name="iter",
    inputs=Inputs(
        parameters={"iter": InputParameter(value=0), "limit": InputParameter(value=3)}
    ),
)

**Note**

1. We are using `dflow.Steps` not `dflow.Step`. 

2. `Steps` is essentially an `OPTemplate` ([See source code](https://github.com/deepmodeling/dflow/blob/master/src/dflow/steps.py)). Using this function, we can construct more complicated workflow. For instance, we are using `dflow.Steps` to create a while loop. 

In [5]:
addition = Step(
    name="add", template=plus1, parameters={"iter": steps.inputs.parameters["iter"]}
)

In [7]:
loop = Step(
    name="loop",
    template=steps,
    parameters={"iter": addition.outputs.parameters["iter"]},
    when="%s < %s"
    % (addition.outputs.parameters["iter"], steps.inputs.parameters["limit"]),
)

This step use steps as its template (note that Steps is a subclass of OPTemplate), meanwhile the steps it used contains this step,which gives a recursion. The recursion will stop when the "when" condition is satisfied (after 10 loops in this example)

In [8]:
steps.add(addition)
steps.add(loop)

In [ ]:
wf = Workflow("recurse", steps=steps)
wf.submit();